<h1 align= center> Taller 1: Modelos de Regresión para la Predicción de Demanda de Bicicletas Compartidas</h1>

### Integrantes

- Axel Mondaca
- Sebastían Hernández

<h4 align = center> Importación de librerias <h4>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
from prettytable import PrettyTable

In [20]:
df = pd.read_csv ('bike_sharing_data.csv')
df

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0000,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17374,17375,2012-12-31,1,1,12,19,0,1,1,2,0.26,0.2576,0.60,0.1642,11,108,119
17375,17376,2012-12-31,1,1,12,20,0,1,1,2,0.26,0.2576,0.60,0.1642,8,81,89
17376,17377,2012-12-31,1,1,12,21,0,1,1,1,0.26,0.2576,0.60,0.1642,7,83,90
17377,17378,2012-12-31,1,1,12,22,0,1,1,1,0.26,0.2727,0.56,0.1343,13,48,61


<h2 align = center> Ingeniería de características y exploración </h2>

Apartado de división de de datos y extracción de características temporales

In [33]:
df['day'] = df['dteday'].str.split('-').str[2].astype(int)

train = df[df['day'] <=14].copy()
val = df[df['day'] >= 15 & (df['day'] <= 21)].copy()
test = df[df['day'] >= 22].copy()

df['year'] = df['dteday'].str.split('-').str[0].astype(int)
df['is_weekend'] = df['weekday'].isin([5, 6]).astype(int)
df['is_rush_hour'] = df['hr'].isin([7, 8, 9, 17, 18, 19]).astype(int)
df['hour_category'] = pd.cut(df['hr'], 
                            bins=[0, 6, 12, 18, 24],
                            labels=[1, 2, 3, 4], right=False)
df

for d in [train, val, test]:
    d.sort_values('dteday', inplace=True)
    d.reset_index(drop=True, inplace=True)

print("Sizes:", len(train), len(val), len(test))
display(train.describe(include='all'))

Sizes: 8026 17379 5345


,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,day,year,is_weekend,is_rush_hour,hour_category
count,8026.000000,8026,8026.000000,8026.000000,8026.000000,8026.000000,8026.000000,8026.000000,8026.000000,8026.000000,8026.000000,8026.000000,8026.000000,8026.000000,8026.000000,8026.000000,8026.000000,8026.000000,8026.000000,8026.000000,8026.000000,8026.0
unique,NaN,336,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
top,NaN,2012-12-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
freq,NaN,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016.0
mean,8471.238973,NaN,2.505607,0.501495,6.517568,11.538874,0.026788,3.002492,0.686768,1.411039,0.489958,0.470477,0.616605,0.189599,35.340892,154.402816,189.743708,7.496885,2011.501495,0.286319,0.251184,NaN
std,5028.466131,NaN,1.116640,0.500029,3.446648,6.915569,0.161473,2.001182,0.463837,0.636480,0.196756,0.175227,0.196482,0.122830,48.925630,150.940044,180.277213,4.032182,0.500029,0.452069,0.433721,NaN
min,1.000000,NaN,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.020000,0.015200,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,2011.000000,0.000000,0.000000,NaN
25%,4269.250000,NaN,2.000000,0.000000,4.000000,6.000000,0.000000,1.000000,0.000000,1.000000,0.340000,0.333300,0.460000,0.104500,4.000000,35.000000,41.000000,4.000000,2011.000000,0.000000,0.000000,NaN
50%,8657.500000,NaN,3.000000,1.000000,7.000000,12.000000,0.000000,3.000000,1.000000,1.000000,0.480000,0.469700,0.610000,0.164200,16.000000,117.000000,143.000000,7.000000,2012.000000,0.000000,0.000000,NaN
75%,13011.750000,NaN,4.000000,1.000000,10.000000,18.000000,0.000000,5.000000,1.000000,2.000000,0.660000,0.621200,0.780000,0.253700,48.000000,221.000000,282.000000,11.000000,2012.000000,1.000000,1.000000,NaN


<h4 align= center> Variable objetivo: Count </h4>